In [2]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import UnitaryGate, HGate, PhaseGate, RZGate, RYGate
from qiskit.synthesis import OneQubitEulerDecomposer

In [4]:
#implement Toffoli Gate 
def toffoli(qc, c0, c1, t0):
    """
    Implement the Toffoli gate according to Figure 4.9 in Nielsen and Chuang's textbook.
    """
    qc.h(t0)
    qc.cx(c1, t0)
    qc.tdg(t0)
    qc.cx(c0, t0)
    qc.t(t0)
    qc.cx(c1, t0)
    qc.tdg(t0)
    qc.cx(c0, t0)
    qc.t(t0)
    qc.h(t0)
    qc.tdg(c1)
    qc.cx(c0, c1)
    qc.tdg(c1)
    qc.cx(c0, c1)
    qc.t(c0)
    qc.s(c1)


#implement single-controlled-U gate
def single_controlled_U(qc, control_index, target_index, U):
    """
    Implement the single controlled U gate according to Figure 4.6 in Nielsen and Chuang's textbook.
    """
    if not isinstance(U, np.ndarray) or U.shape != (2, 2):
        raise ValueError("U must be a 2x2 numpy array")
    if not np.allclose(U @ U.conj().T, np.eye(2)):
        raise ValueError("U must be unitary")
    if control_index == target_index:
        raise ValueError("Control and target indices must be different")
    if control_index < 0 or target_index < 0 or control_index >= qc.num_qubits or target_index >= qc.num_qubits:
        raise ValueError("Qubit indices out of range")

    decomposer = OneQubitEulerDecomposer(basis='ZYZ')
    gamma, beta, delta, alpha = decomposer.angles_and_phase(U)

    #apply C
    qc.append(RZGate((delta-beta)/2), [target_index])

    #apply CNOT
    qc.cx(control_index, target_index)

    #apply B
    qc.append(RZGate(-(delta+beta)/2), [target_index])
    qc.append(RYGate(-gamma/2), [target_index])

    #apply CNOT
    qc.cx(control_index, target_index)

    #apply A
    qc.append(RYGate(gamma/2), [target_index])
    qc.append(RZGate(beta), [target_index])

    #apply Phase P
    qc.append(PhaseGate(alpha), [control_index])


#implement multi-controlled-U gate
def multi_controlled_U_with_ancilla(circuit, control_qubits, target_qubit, U, ancilla_qubits=None):
    """
    Implement multi-controlled-U gate according to the Figure 4.10. in Nielsen and Chuang's book.
    
    Args:
        circuit: Existing QuantumCircuit to modify.
        control_qubits: List of indices for control qubits.
        target_qubit: Index of the target qubit.
        U: 2x2 unitary matrix (np.ndarray).
        ancilla_qubits: Optional list of ancilla qubit indices; if None, new ancillas are added.
    
    Returns:
        QuantumCircuit: Modified circuit with multi-controlled U gate.
    """
    n = len(control_qubits)
    if n < 1:
        raise ValueError("At least one control qubit is required")
    
    # Validate inputs
    if target_qubit in control_qubits:
        raise ValueError("Target qubit cannot be a control qubit")
    if not all(0 <= q < circuit.num_qubits for q in control_qubits + [target_qubit]):
        raise ValueError("Qubit indices out of range")
    
    # Determine number of ancillas needed
    num_ancillas_needed = max(0, n - 1)
    
    # If ancilla_qubits not provided, add new ancilla qubits
    if ancilla_qubits is None:
        if num_ancillas_needed > 0:
            ancilla_reg = AncillaRegister(num_ancillas_needed, "a")
            new_circuit = QuantumCircuit(*circuit.qregs, ancilla_reg)
            new_circuit.compose(circuit, inplace=True)
            circuit = new_circuit
            ancilla_qubits = [circuit.num_qubits - num_ancillas_needed + i for i in range(num_ancillas_needed)]
    else:
        if len(ancilla_qubits) < num_ancillas_needed:
            raise ValueError(f"Too few ancilla qubits provided: {len(ancilla_qubits)} < {num_ancillas_needed}")
        if not all(0 <= q < circuit.num_qubits for q in ancilla_qubits):
            raise ValueError("Ancilla qubit indices out of range")
        if any(q in control_qubits + [target_qubit] for q in ancilla_qubits):
            raise ValueError("Ancilla qubits must be distinct from control and target qubits")
    
    # Base case: n=1
    if n == 1:
        single_controlled_U(circuit, control_qubits[0], target_qubit, U)
        return circuit
    
    # Step 1: Compute partial ANDs
    toffoli(circuit, control_qubits[0], control_qubits[1], ancilla_qubits[0])
    circuit.barrier()
    for i in range(2, n):
        toffoli(circuit, control_qubits[i], ancilla_qubits[i-2], ancilla_qubits[i-1])
        circuit.barrier()
    
    # Step 2: Apply single-controlled U
    single_controlled_U(circuit, ancilla_qubits[-1], target_qubit, U)
    circuit.barrier()
    
    # Step 3: Uncompute ancillas
    for i in reversed(range(2, n)):
        toffoli(circuit, control_qubits[i], ancilla_qubits[i-2], ancilla_qubits[i-1])
        circuit.barrier()
    toffoli(circuit, control_qubits[0], control_qubits[1], ancilla_qubits[0])
    
    return circuit


In [22]:
def generate_random_unitary():
    """
    Generate a random 2x2 unitary matrix using Pauli matrices.
    
    Returns:
        U (np.ndarray): A 2x2 unitary matrix.
    """
    # Pauli matrices and identity
    I = np.array([[1, 0], [0, 1]], dtype=complex)
    sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
    sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)
    
    # Random global phase alpha in [0, 2pi)
    alpha = np.random.uniform(0, 2 * np.pi)
    
    # Random rotation angle theta in [0, pi]
    theta = np.random.uniform(0, np.pi)
    
    # Random unit vector n = (n_x, n_y, n_z)
    # Generate random point on unit sphere using spherical coordinates
    phi = np.random.uniform(0, 2 * np.pi)  # Azimuthal angle
    cos_theta_n = np.random.uniform(-1, 1)  # cos of polar angle
    sin_theta_n = np.sqrt(1 - cos_theta_n**2)
    n_x = sin_theta_n * np.cos(phi)
    n_y = sin_theta_n * np.sin(phi)
    n_z = cos_theta_n
    
    # Verify unit vector
    assert np.isclose(n_x**2 + n_y**2 + n_z**2, 1, atol=1e-8), "Not a unit vector"
    
    # Construct n . sigma
    n_dot_sigma = n_x * sigma_x + n_y * sigma_y + n_z * sigma_z
    
    # Compute U = e^(i alpha) * [cos(theta/2) I - i sin(theta/2) (n . sigma)]
    U = np.exp(1j * alpha) * (
        np.cos(theta / 2) * I - 1j * np.sin(theta / 2) * n_dot_sigma
    )
    
    # Verify unitarity
    assert np.allclose(U.conj().T @ U, I, atol=1e-8), "Matrix is not unitary"
    
    return U

# Example usage
#if __name__ == "__main__":
# Generate a random unitary matrix
#    U = generate_random_unitary()
#    print("Random 2x2 unitary matrix:")
#    print(U)
    
# Verify unitarity
#    print("UU_dagger (should be identity):")
#    print(U.conj().T @ U)

In [20]:

# Test function 
def test_multi_controlled_U_against_control():
    """
    Test multi_controlled_U_with_ancilla against Qiskit's UnitaryGate.control() for an arbitrary unitary U.
    """
    # Define arbitrary unitary matrix
    #U = np.array([[1, 1j], [1j, 1]]) / np.sqrt(2)
    #randomly generate a unitary matrix.
    U = generate_random_unitary()
    
    print("Testing multi_controlled_U_with_ancilla against Qiskit's UnitaryGate.control()")
    print(f"Unitary U:\n{U}\n")
    
    
    # Test for n=1, 2, 3, 4 control qubits
    for n in [1, 2, 3, 4]:
        print(f"Testing with n={n} control qubits:")
        num_ancillas = max(0, n - 1)
        total_qubits_custom = n + 1 + num_ancillas  # controls + target + ancillas
        total_qubits_qiskit = n + 1  # controls + target
        
        # Test inputs
        test_inputs = [
            (f"|{'0' * n}0>", lambda circ: None),  # All controls off
            (f"|{'1' * n}0>", lambda circ: circ.x(list(range(n)))),  # All controls on
            (f"|{'+' * n}0>", lambda circ: circ.h(list(range(n))))  # Controls in |+>
        ]
        
        for input_state, prep_func in test_inputs:
            print(f"Testing input {input_state}:")
            # Custom circuit
            qr_custom = QuantumRegister(n + 1, "q")
            circuit_custom = QuantumCircuit(qr_custom)
            if prep_func:
                prep_func(circuit_custom)
            circuit_custom = multi_controlled_U_with_ancilla(
                circuit_custom,
                control_qubits=list(range(n)),
                target_qubit=n,
                U=U
            )
            state_custom = Statevector(circuit_custom).data
            
            # Project custom statevector to ancilla |0> subspace
            dim_qiskit = 2 ** total_qubits_qiskit
            state_custom_projected = np.zeros(dim_qiskit, dtype=complex)
            for i in range(dim_qiskit):
                full_index = i  # Ancillas are 0 at highest indices
                state_custom_projected[i] = state_custom[full_index]
            
            # Create Statevector object for projected state
            state_custom_projected_sv = Statevector(state_custom_projected)
            
            # Qiskit circuit
            qr_qiskit = QuantumRegister(n + 1, "q")
            circuit_qiskit = QuantumCircuit(qr_qiskit)
            if prep_func:
                prep_func(circuit_qiskit)
            unitary_gate = UnitaryGate(U, label="U")
            controlled_gate = unitary_gate.control(n)
            circuit_qiskit.append(controlled_gate, list(range(n + 1)))
            state_qiskit = Statevector(circuit_qiskit)
            
            # Compare statevectors
            match = np.allclose(state_custom_projected, state_qiskit.data, atol=1e-8)
            print(f"Statevector match: {match}")
            
            # Output statevectors using draw("latex_source")
            print("Custom statevector (projected):")
            print(f"${state_custom_projected_sv.draw('latex_source')}$")
            print("Qiskit statevector:")
            print(f"${state_qiskit.draw('latex_source')}$")
            
            # Output qubit counts
            print(f"Custom circuit qubits: {circuit_custom.num_qubits} (expected: {total_qubits_custom})")
            print(f"Qiskit circuit qubits: {circuit_qiskit.num_qubits} (expected: {total_qubits_qiskit})\n")

if __name__ == "__main__":
    test_multi_controlled_U_against_control()

Testing multi_controlled_U_with_ancilla against Qiskit's UnitaryGate.control()
Unitary U:
[[-0.59077658-0.65168427j  0.26525936-0.39487734j]
 [-0.35980896-0.31117223j -0.39849739+0.78416145j]]

Testing with n=1 control qubits:
Testing input |00>:
Statevector match: True
Custom statevector (projected):
$ |00\rangle$
Qiskit statevector:
$ |00\rangle$
Custom circuit qubits: 2 (expected: 2)
Qiskit circuit qubits: 2 (expected: 2)

Testing input |10>:
Statevector match: True
Custom statevector (projected):
$(-0.5907765817 - 0.6516842702 i) |01\rangle+(-0.3598089609 - 0.3111722259 i) |11\rangle$
Qiskit statevector:
$(-0.5907765817 - 0.6516842702 i) |01\rangle+(-0.3598089609 - 0.3111722259 i) |11\rangle$
Custom circuit qubits: 2 (expected: 2)
Qiskit circuit qubits: 2 (expected: 2)

Testing input |+0>:
Statevector match: True
Custom statevector (projected):
$\frac{\sqrt{2}}{2} |00\rangle+(-0.4177421271 - 0.4608103666 i) |01\rangle+(-0.2544233562 - 0.220031991 i) |11\rangle$
Qiskit statevector:
